In [65]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =['%', "‘", 'chahie|', "\\", '/', 'tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        for i in data['Freq']:
            print(i)
        #for i in data['Pairs']:
         #   print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''bhojana, arama, ayurvedika malishom aura lambi sairom ke satha anivarya kharidadari, apaki yaham ki yatra ko yadagara aura eka vastavika avakasha banaegi | agasta / sitambara ke mahine mem manaya jane vala onama, kerala ke sarvadhika mahatvapurna tyoharom mem se eka hai | onama bahuta si umanga jagata hai aura shahara apani khushi khulakara vyakta karata hai | hara kisi ke dalana mem manohara akritiyom mem pushpa sajae jate haim aura sabhi jagahom para sanskritika karyakrama ayojita hote haim | hara eka sthaniya mandira ko prakashita kiya jata hai jisase pahale se hi manohara sanrachana ki sundarata aura badha़ jati hai |
yaha utsava eka ranga-birange samaroha mem samapta hota hai jo sthaniya logom tatha paryatakom ko samana rupa se akarshita karata hai | bharata ki kuchha paramparagata shastriya nritya shailiyom ka mancha, yaha nritya utsava prativarsha 21 se 27 pharavari taka kanakakunnu pailesa maidana mem sthita nishagandhi khuli natyashala mem ayojita kiya jata hai | bharatanatyama, kathaka, kathakali, kuchipuड़i, manipuri, aura mohiniattama se odisi taka, vibhinna rupom mem lavanya aura lalitya ka hissa banane ka avasara hai nishagandhi nritya utsava | eka sundara aura anuthi sankalpana, gramama, 14 se 23 janavari taka ayojita ki jati hai |
kovalama mem ayojita 10 dina lambe isa asadharana utsava ke samaya samudra - tata para sampurna kerala gamva ka punarnirmana kiya jata hai| ratonrata utha ane vale nalukettu ka parisara mele ka ayojana sthala bana jata hai, aura rajya ki paramparagata kala aura shilpa ko darshita karata hai| 5 se 11 apraila taka kanakakunnu mahala ke parisara mem manae jane vale svada bhojanotsava (phlevara phuda phestivala) mem kerala bhara ke mashahura hotala bhaga lete haim| yaham atyadhika pratiyogita hoti hai jaham pratiyogi hotala sansara ke sabhi jagahom ke bhojanom ko pradarshita karate hue bhoja pesha karate haim | yadi apa eka bhojana premi haim (to) svada bhojanotsava (phlevara phuda phestivala) apaki sacha hui priya kalpana hai| trivendrama sabhi bada़e pradeshika tatha rashtriya rajamargom dvara shesha dakshina bharata se achchhi taraha se juda़a hua hai | pashchima eshiya, shrilanka aura maladiva se kuchha sidhi uda़anom ke satha, trivendrama dakshina bharata ka eka mahatvapurna antarrashtriya praveshadvara hai | indiyana eyaralainsa aura jeta eyaraveja़ donom mumbai, dilli aura chennai ki niyamita uda़anom se trivendrama ko desha ke shesha bhagom se joda़ti haim | madhyapradesha bharata ka sabase bada़a rajya aura desha ka bhaugolika kendra sthala hai | rajya ka adhikatara bhaga eka uncha pathara hai aura grishmaritu mem bahuta shushka tatha garama ho sakata hai | aitihasika rupa se malava ke nama se jane jane vale madhyapradesha para bharatiya itihasa ki lagabhaga sabhi sthitiyam apani chhapa chhoda़ gai haim |
isa rajya mem aba bhi kai arya - purva gonda aura bhila janajatiya loga haim, parantu bahusankhyaka janata dvara hindi bole jane aura hindu dharma manane ke satha hi madhyapradesha adhikanshataya hinda - arya (indo - aryana) hai | madhyapradesha ke kuchha akarshana haim - khajuraho aura kanha rashtriya udyana (apane baghom ke lie prasiddha) | madhyapradesha gvaliyara kile ke lie prasiddha hai | bhavya bahari divarom aura sundara antarika mahalom ke satha, balue patthara ka yaha ajeya kila madhyapradesha para varchasva rakhata hai| vishala sabhagrihom aura nakkashidara patthara ki ada़om se yukta mana mandira mahala, raja manasinha dvara banavaya gaya tha | gujari mahala 15vim shatabdi ke vastushilpa ka eka ashcharya hai; yaha mahala raja manasinha ka apani priya rani nriganayani ke lie upahara tha | bhagavana vishnu ko samarpita do akarshaka mandira, sasa bahu ka mandira aura teli ka mandira, vastushilpa shailiyom ke advitiya sammilana se bane haim | ajakala jaya vilasa pailesa aura sangrahalaya shahi sindhiya parivara ka nivasa ka sthana hai; bhavya darabara haॉla sahita, chitrayavanikaom aura puratana pharnichara se susajjita yaha mahala eka italavi sanrachana hai |
tanasena ki kabra usa mahana gayaka ki smaraka hai jo shahanshaha akabara ke darabara ke nau ratnom mem se eka tha | apa vidhanasabha , taigora smaraka aura rashtriya lekhagara ke samane se gada़i dvara bhi gujara sakate haim | bhojapura raja bhoja dvara sthapita kiya gaya tha aura yaham shiva mandira ke dhvansavashesha haim |
kancha mandira, kancha ke motiyom mem taiyara kiya gaya jaina punya sthala, lalabaga mahala aura naya mahala dekhem | 950 se 1050 isvi ke bicha vishva prasiddha khajuraho mandira chandela rajaom dvara banavaye gaye the | sabase mahatvapurna haim devi kali ko samarpita chaunsatha yogini mandira, mahadeva mandira, chitragupta mandira athava apane ekadashamukhi vishnu ki sundara murti ke satha bharataji mandira, vishvanatha aura nandi mandira, lakshmana mandira, shiva ka visha mandira, jo sabase adhika vishala aura vishishta mandira hai|
dekhem purvi samuha ke mandirom ko jisamem vidyamana hai parasvanatha mandira - khajuraho mem bacha hua ekamatra jaina mandira jisake garbhagriha ki bahya divarom para uttama murtikala hai- evam vishnu ko samarpita javeri mandira | isa samuha mem dusare mandira ve haim jo brahma, yamuna aura adinatha ko samarpita haim |
ananda ka yaha nagara raja baja bahadura ke apani sangini rani rupamati ke prati prema ki kahaniyom ke lie prasiddha hai | mandu, 13 vim shatabdi mem hindu paramara rajaom ki mula rajadhani, bada mem malava ke sultanom dvara jita liya gaya tha | dekhem mahala, snanagriha, kheme, pratyeka aphagani vastushilpa shaili ke ratna, masjidom aura atisundara samadhiyom ko | sanchi ka vishala stupa bharata ki sabase purani prastara sanrachana hai | stupa mem bhavyata se tarashe gae praveshadvara ya torana haim | bauddha kala ke sundaratama udaharanom mem se sanchi eka hai | chara pramukha dvarom mem pratyeka adhyayana yogya hai | ashoka ke stambhom ke sabase sundara udaharanom mem se eka ashoka stambha, dakshina mukhya dvara ke nikata sthita hai | gupta mandira, 5vim shatabdi isvi ka eka mandira, bharata ki mandira vastukala ke saba se purane jnata udaharanom mem se eka hai | bhopala se 68 kimi. dura, sanchi ko tisari shatabdi isa purva se hi lagabhaga sare bauddha vastushilpiya rupom ke sabase uttama namune stupa, chaitya, mandira aura matha apane yaham hone ka gaurava prapta hai | kshetra mem 156 varga kimi.,shivapuri ka madhava rashtriya udyana sala bhara khula rahata hai |
jangali pahada़iyom ke vividhatapurna bhubhaga ke satha yaha eka vibhinna prakara ke vanya jivana ko dekhane ke prachura avasara pradana karata hai | pramukha prajati jo isa udyana mem nivasa karati hai vaha hai hirana, unamem se sabase ama haim naja़uka chinkara, bharatiya gaija़ela aura chitala | madhava rashtriya udyana mem dusari ama jatiyam haim nilagaya, sambhara, krishnanriga (blaikabaka), slaॉtha biyara, chita (leparda) aura langura |
ujjaina bharata ke sabase pavitra aura purane shaharom mem se eka hai aura ashoka ke raja mem bauddha dharma ke prasara mem yaha eka mahattvapurna shahara tha | onkareshvara mem sthita jyotirlinga ke liye prasiddha, yaha bharata ke sata pavitra nagarom mem se eka hai | 12 varshom mem eka bara, vishala ‘kumbha mela’ utsava ujjaina mem ayojita hota hai | mahakala ka prachina punya sthala isake akarshanom mem se eka hai |
1531 aura 1783 ke bicha orachha bundela rajaputom ki rajadhani thi | bundela rajaputom dvara banavae gae mahala aura mandira apani adhikansha puratana paripurnata ko kayama rakhe hue haim | kuchha mahala divara para bane bhittichitrom se alankrita kie gae the jo bundela chitrakari shaili ke sarvottama pallavana ka pratinidhitva karate haim | kile ke sankula ke andara jahangira mahala 17vim shatabdi mem shahanshaha jahangira ke agamana ki yada mem bana tha | jahangira mahala ki tikhi rekhaem usaki saunya chhatariyom aura jaफ़ri ke kama se prati santulita haim | rajamahala: mahala ka antarika bhaga vividha dharmika vishaya vastuom para tejasvi rangom se chitrita kiya gaya hai | bahuta unche shikharom ke satha ramaraja mandira eka asadharana mandira hai, jo kabhi pahale eka mahala tha; yaham bhagavana rama ki eka raja ki taraha upasana hoti hai | chaturbhaja mandira patthara ke eka vishalakaya chabutare para bana hua hai; upara ki ora khada़i sidha़iyom dvara mandira taka pahuncha jata hai | lakshminarayana mandira ke antarika bhaga mem orachha ki bhittichitra kalaom mem se sabase utkrishta chitra haim | kavayitri aura sangitajna raya pravina, jinhonne 17vim sadi mem raja indramani ko mohita kara liya tha, ke liye intom se bana mahala, raya pravina mahala bhudrishyi udyanom mem sthita hai | mukhya marga para bhopala rela dvara bharata mem pramukha shaharom se juda़a hai | niyamita basa sevaem bhopala ko sanchi, indaura, ujjaina, mandu aura panchamadha़i se joda़ti haim | kanha rashtriya udyana (jo aba adhikarika rupa se kanha bagha sanrakshita kshetra ke nama se jana jata hai) madhya bharatiya parvatiya kshetrom ki satapuda़a pahada़iyom ke purvi khanda, maikala shrrinkhala mem sthita hai | yaha udyana madhyapradesha rajya mem jabalapura se 160 kimi. (100 mila) dakshina-purva mem pada़ta hai | kanha ke nama ki vyutpatti hui hai kanhara se jo ki ghati ki talahati mem pai jane vali chikani mitti ke liye sthaniya shabda hai, ya kanva se, jo ki eka samaya vaham vana ke eka gamva mem rahane vale eka sadhu purusha the | do nadi - ghatiyam udyana ki bhaugolika sthiti ki mahatvapurna visheshataem haim : pashchima mem banjara aura purva mem halona | ye donom nadiyam narmada ki sahayaka nadiyam haim, jo ki udyana ke pashchimi praveshadvara ke 64 kilomitara (40 mila) uttara-pashchima mem mandala ke jila mukhyalaya nagara se hokara bahati hai | kanha ki ghatiyam sthaniya bhasha mem dadara kahalane vale patharom vale pahaड़om se samavarita haim | prabandhana ke uddeshyom ke lie, udyana ko 5 kshetrom mem banta gaya hai : kisali, kanha aura mukki pashchimi khanda mem aura bhainsaghata aura supakhara purvi khanda mem | kisali, kanha aura mukki shrrinkhalaom ke andara, kuchha sada़kem aura sthana bhi paryatakom ke lie banda haim | yaha likhe jane taka (1998 madhya), kanha rashtriya udyana ke darshakom ke liye khule hue kshetra (227 varga kimi.) ka pratishata abhyantara kshetra ka 25% hai | kanha rashtriya udyana samanyataya uttama rakha-rakhava ki hui saड़kom ke vyapaka jala (700 kimi. se kinchita adhika) se juड़a hai |
kanha rashtriya udyana 1 navambara se 30 juna taka pratidina khula rahata hai, yadyapi prabandhana ke pasa juna anta ke purva varsha hone para jaldi banda karane ka vikalpa arakshita hai | kanha rashtriya udyana ke khulane ka samaya anumanatah mausami utara - chaढ़ava, satha hi vibhinna mausamom mem dina ki lambai ke sammata nirdharita kiya jata hai | kanha rashtriya udyana mem madhyanha avakasha sadaiva hota hai, taki karmachari bhojana aura vishrama kara sakem aura janavarom ko bhi thoड़a arama mila sake | 1000 isvi ke asa-pasa bane khajuraho (kha-ju-ra-ho) ke ye mandira vishva ke lie ratyatmaka prema ke bharata ke amara upahara haim | jivana ke rangina, sukhada, tathapi udatta sara ki murta rupa, khajuraho ki daiviya pratimaem svayam jivana ke liye upaharasvarupa haim | kisi samaya jumate hue khajura ke vrikshom, jinase khajuraho shahara ka namakarana hua, se abhirakshita yaha shahara kala aura murtishilpa ka adhyayana hai | vastava mem, chandela mandira aja bhi aghata, ashcharya, abhibhuti aura prashansa sahita hajaro anya bhavanaom ko jagane vale apane vividha akarshaka murtarupom sahita khajuraho ka kendra haim | hindu-arya (indo-aryana) vastukala ka eka utkrishta udaharana, mandira ki divarom para kala jatilata aura sadagi ka eka parishkrita sanyojana hai | khajuraho shahara apani utpatti 10vim shatabdi se batata hai jaba yaha chandela rajavansha ka gadha़ tha |
aja ka khajuraho vanijyikarana ke adyatana smarakom, samanya sabhyata ke pratikom - hotalom, bhojanalayom aura dukanom - se ज़yada parichita hai| mandirom ke rupa mem purna bhavyatashali itihasa apani saunya drishti se usa nagara ko dekha raha hai jo apane shasakom ki dena ke bala para aja taka jivanta hai|
nissandeha, yaha shahara pratyeka varsha marcha mem nrityotsava ke daurana jivanta ho uthata hai, jaba shastriya nritya ke purodha, biraju maharaja aura kelucharana mahapatra aura unake shishya mandira ki natakiya prishthabhumi mem prastuti dete haim | nissandeha, khajuraho eka aisa bharatiya gantavya nahim hai jise apa asani se chhoda़ sakate haim | chandela vastukala spashtatah apane charama para thi jaba ye mandira banae gae the | devom aura deviyom, shuravira yoddhaom, pashuom aura logom ke drishyom ko ankita karati ye murtiyam aura utkirnana bahudha premalapa mem tallina dikhai jati haim | kuchha ko abhibhuta aura anya ko aghati lagane vali vishaya vasana aura ratyatmakata ki yaha prastuti eka bara bhi ashlila nahim lagati | isa kala ki sundarata mandira kala rupa ki shuddhata ko samvriddha karati hai | ye mandira eka uttunga shikhara mem samapta hone vali (aisi) minarom ki eka shrrinkhala haim, jinamem se pratyeka anya ki apeksha adhika unchi hai | yaha ashcharyajanaka hai ki ye khanda ekadusare ke satha sahajata se juda़e hue haim jabaki inake banate samaya gara nahim tha | lakshmana mandira karigari mem jatila hai, kandariya mahadeva, jaham andaruni bhaga mem bhavyata se tarashi gayi 872 murtiyam haim, sabase baड़a aura vastushilpa ki drishti se sabase purna hai, mahadeva mandira sabase chhota hai; devi jagadamba mandira sarvaprathama vishnu ko samarpita tha, tatpashchata parvati aura anta mem kali ko samarpita kiya gaya| unnisavim shatabdi ke dusare ardhda mem banae gae sabase prabhavi mahalom mem se eka lalagadha़ mahala bikanera ke charom ora nishthura thara marusthala ke rukhe evam vishama satya ko nakarata hai| lalagadha़ mahala bharatiya-arabi shilpa ka sabase purna rupa se sanyukta udaharana tha| 
apa ganga goldana jubali nyujiyama mem pakki mitti ke samana, paramparagata rajasthani vadyayantra aura raॉyala bikanera rela ke eka chhote pratirupa, prachina galichom evam rajasi kapada़om ka ananda le sakate haim| 
badashaha akabara ke maranasanna hone ke karana rajakumara jahaॉngira dvara rayasinha ko dilli jane ke lie di ja rahi rajakiya ajna bhi dekha sakate haim| kemala bridinga pharma shahara se matra 8 kimi. dura hai isa sarakari unta palana kshetra mem apa ‘marusthala ke jahaja’ ke bare mem bahuta kuchha jana pate haim| 
apako pata lagata hai ki kana para balom vala lambe bali vala untha, bikanera ki prajati hai jo ki apani takata ke lie prasidhda hai| halake ranga vale untha jaisalamera se haim aura apani gati ke karana bahumulya haim|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

9
7
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [66]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =['%', "‘", 'chahie|', "\\", '/', 'tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        #for i in data['Freq']:
         #   print(i)
        for i in data['Pairs']:
            print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''bhojana, arama, ayurvedika malishom aura lambi sairom ke satha anivarya kharidadari, apaki yaham ki yatra ko yadagara aura eka vastavika avakasha banaegi | agasta / sitambara ke mahine mem manaya jane vala onama, kerala ke sarvadhika mahatvapurna tyoharom mem se eka hai | onama bahuta si umanga jagata hai aura shahara apani khushi khulakara vyakta karata hai | hara kisi ke dalana mem manohara akritiyom mem pushpa sajae jate haim aura sabhi jagahom para sanskritika karyakrama ayojita hote haim | hara eka sthaniya mandira ko prakashita kiya jata hai jisase pahale se hi manohara sanrachana ki sundarata aura badha़ jati hai |
yaha utsava eka ranga-birange samaroha mem samapta hota hai jo sthaniya logom tatha paryatakom ko samana rupa se akarshita karata hai | bharata ki kuchha paramparagata shastriya nritya shailiyom ka mancha, yaha nritya utsava prativarsha 21 se 27 pharavari taka kanakakunnu pailesa maidana mem sthita nishagandhi khuli natyashala mem ayojita kiya jata hai | bharatanatyama, kathaka, kathakali, kuchipuड़i, manipuri, aura mohiniattama se odisi taka, vibhinna rupom mem lavanya aura lalitya ka hissa banane ka avasara hai nishagandhi nritya utsava | eka sundara aura anuthi sankalpana, gramama, 14 se 23 janavari taka ayojita ki jati hai |
kovalama mem ayojita 10 dina lambe isa asadharana utsava ke samaya samudra - tata para sampurna kerala gamva ka punarnirmana kiya jata hai| ratonrata utha ane vale nalukettu ka parisara mele ka ayojana sthala bana jata hai, aura rajya ki paramparagata kala aura shilpa ko darshita karata hai| 5 se 11 apraila taka kanakakunnu mahala ke parisara mem manae jane vale svada bhojanotsava (phlevara phuda phestivala) mem kerala bhara ke mashahura hotala bhaga lete haim| yaham atyadhika pratiyogita hoti hai jaham pratiyogi hotala sansara ke sabhi jagahom ke bhojanom ko pradarshita karate hue bhoja pesha karate haim | yadi apa eka bhojana premi haim (to) svada bhojanotsava (phlevara phuda phestivala) apaki sacha hui priya kalpana hai| trivendrama sabhi bada़e pradeshika tatha rashtriya rajamargom dvara shesha dakshina bharata se achchhi taraha se juda़a hua hai | pashchima eshiya, shrilanka aura maladiva se kuchha sidhi uda़anom ke satha, trivendrama dakshina bharata ka eka mahatvapurna antarrashtriya praveshadvara hai | indiyana eyaralainsa aura jeta eyaraveja़ donom mumbai, dilli aura chennai ki niyamita uda़anom se trivendrama ko desha ke shesha bhagom se joda़ti haim | madhyapradesha bharata ka sabase bada़a rajya aura desha ka bhaugolika kendra sthala hai | rajya ka adhikatara bhaga eka uncha pathara hai aura grishmaritu mem bahuta shushka tatha garama ho sakata hai | aitihasika rupa se malava ke nama se jane jane vale madhyapradesha para bharatiya itihasa ki lagabhaga sabhi sthitiyam apani chhapa chhoda़ gai haim |
isa rajya mem aba bhi kai arya - purva gonda aura bhila janajatiya loga haim, parantu bahusankhyaka janata dvara hindi bole jane aura hindu dharma manane ke satha hi madhyapradesha adhikanshataya hinda - arya (indo - aryana) hai | madhyapradesha ke kuchha akarshana haim - khajuraho aura kanha rashtriya udyana (apane baghom ke lie prasiddha) | madhyapradesha gvaliyara kile ke lie prasiddha hai | bhavya bahari divarom aura sundara antarika mahalom ke satha, balue patthara ka yaha ajeya kila madhyapradesha para varchasva rakhata hai| vishala sabhagrihom aura nakkashidara patthara ki ada़om se yukta mana mandira mahala, raja manasinha dvara banavaya gaya tha | gujari mahala 15vim shatabdi ke vastushilpa ka eka ashcharya hai; yaha mahala raja manasinha ka apani priya rani nriganayani ke lie upahara tha | bhagavana vishnu ko samarpita do akarshaka mandira, sasa bahu ka mandira aura teli ka mandira, vastushilpa shailiyom ke advitiya sammilana se bane haim | ajakala jaya vilasa pailesa aura sangrahalaya shahi sindhiya parivara ka nivasa ka sthana hai; bhavya darabara haॉla sahita, chitrayavanikaom aura puratana pharnichara se susajjita yaha mahala eka italavi sanrachana hai |
tanasena ki kabra usa mahana gayaka ki smaraka hai jo shahanshaha akabara ke darabara ke nau ratnom mem se eka tha | apa vidhanasabha , taigora smaraka aura rashtriya lekhagara ke samane se gada़i dvara bhi gujara sakate haim | bhojapura raja bhoja dvara sthapita kiya gaya tha aura yaham shiva mandira ke dhvansavashesha haim |
kancha mandira, kancha ke motiyom mem taiyara kiya gaya jaina punya sthala, lalabaga mahala aura naya mahala dekhem | 950 se 1050 isvi ke bicha vishva prasiddha khajuraho mandira chandela rajaom dvara banavaye gaye the | sabase mahatvapurna haim devi kali ko samarpita chaunsatha yogini mandira, mahadeva mandira, chitragupta mandira athava apane ekadashamukhi vishnu ki sundara murti ke satha bharataji mandira, vishvanatha aura nandi mandira, lakshmana mandira, shiva ka visha mandira, jo sabase adhika vishala aura vishishta mandira hai|
dekhem purvi samuha ke mandirom ko jisamem vidyamana hai parasvanatha mandira - khajuraho mem bacha hua ekamatra jaina mandira jisake garbhagriha ki bahya divarom para uttama murtikala hai- evam vishnu ko samarpita javeri mandira | isa samuha mem dusare mandira ve haim jo brahma, yamuna aura adinatha ko samarpita haim |
ananda ka yaha nagara raja baja bahadura ke apani sangini rani rupamati ke prati prema ki kahaniyom ke lie prasiddha hai | mandu, 13 vim shatabdi mem hindu paramara rajaom ki mula rajadhani, bada mem malava ke sultanom dvara jita liya gaya tha | dekhem mahala, snanagriha, kheme, pratyeka aphagani vastushilpa shaili ke ratna, masjidom aura atisundara samadhiyom ko | sanchi ka vishala stupa bharata ki sabase purani prastara sanrachana hai | stupa mem bhavyata se tarashe gae praveshadvara ya torana haim | bauddha kala ke sundaratama udaharanom mem se sanchi eka hai | chara pramukha dvarom mem pratyeka adhyayana yogya hai | ashoka ke stambhom ke sabase sundara udaharanom mem se eka ashoka stambha, dakshina mukhya dvara ke nikata sthita hai | gupta mandira, 5vim shatabdi isvi ka eka mandira, bharata ki mandira vastukala ke saba se purane jnata udaharanom mem se eka hai | bhopala se 68 kimi. dura, sanchi ko tisari shatabdi isa purva se hi lagabhaga sare bauddha vastushilpiya rupom ke sabase uttama namune stupa, chaitya, mandira aura matha apane yaham hone ka gaurava prapta hai | kshetra mem 156 varga kimi.,shivapuri ka madhava rashtriya udyana sala bhara khula rahata hai |
jangali pahada़iyom ke vividhatapurna bhubhaga ke satha yaha eka vibhinna prakara ke vanya jivana ko dekhane ke prachura avasara pradana karata hai | pramukha prajati jo isa udyana mem nivasa karati hai vaha hai hirana, unamem se sabase ama haim naja़uka chinkara, bharatiya gaija़ela aura chitala | madhava rashtriya udyana mem dusari ama jatiyam haim nilagaya, sambhara, krishnanriga (blaikabaka), slaॉtha biyara, chita (leparda) aura langura |
ujjaina bharata ke sabase pavitra aura purane shaharom mem se eka hai aura ashoka ke raja mem bauddha dharma ke prasara mem yaha eka mahattvapurna shahara tha | onkareshvara mem sthita jyotirlinga ke liye prasiddha, yaha bharata ke sata pavitra nagarom mem se eka hai | 12 varshom mem eka bara, vishala ‘kumbha mela’ utsava ujjaina mem ayojita hota hai | mahakala ka prachina punya sthala isake akarshanom mem se eka hai |
1531 aura 1783 ke bicha orachha bundela rajaputom ki rajadhani thi | bundela rajaputom dvara banavae gae mahala aura mandira apani adhikansha puratana paripurnata ko kayama rakhe hue haim | kuchha mahala divara para bane bhittichitrom se alankrita kie gae the jo bundela chitrakari shaili ke sarvottama pallavana ka pratinidhitva karate haim | kile ke sankula ke andara jahangira mahala 17vim shatabdi mem shahanshaha jahangira ke agamana ki yada mem bana tha | jahangira mahala ki tikhi rekhaem usaki saunya chhatariyom aura jaफ़ri ke kama se prati santulita haim | rajamahala: mahala ka antarika bhaga vividha dharmika vishaya vastuom para tejasvi rangom se chitrita kiya gaya hai | bahuta unche shikharom ke satha ramaraja mandira eka asadharana mandira hai, jo kabhi pahale eka mahala tha; yaham bhagavana rama ki eka raja ki taraha upasana hoti hai | chaturbhaja mandira patthara ke eka vishalakaya chabutare para bana hua hai; upara ki ora khada़i sidha़iyom dvara mandira taka pahuncha jata hai | lakshminarayana mandira ke antarika bhaga mem orachha ki bhittichitra kalaom mem se sabase utkrishta chitra haim | kavayitri aura sangitajna raya pravina, jinhonne 17vim sadi mem raja indramani ko mohita kara liya tha, ke liye intom se bana mahala, raya pravina mahala bhudrishyi udyanom mem sthita hai | mukhya marga para bhopala rela dvara bharata mem pramukha shaharom se juda़a hai | niyamita basa sevaem bhopala ko sanchi, indaura, ujjaina, mandu aura panchamadha़i se joda़ti haim | kanha rashtriya udyana (jo aba adhikarika rupa se kanha bagha sanrakshita kshetra ke nama se jana jata hai) madhya bharatiya parvatiya kshetrom ki satapuda़a pahada़iyom ke purvi khanda, maikala shrrinkhala mem sthita hai | yaha udyana madhyapradesha rajya mem jabalapura se 160 kimi. (100 mila) dakshina-purva mem pada़ta hai | kanha ke nama ki vyutpatti hui hai kanhara se jo ki ghati ki talahati mem pai jane vali chikani mitti ke liye sthaniya shabda hai, ya kanva se, jo ki eka samaya vaham vana ke eka gamva mem rahane vale eka sadhu purusha the | do nadi - ghatiyam udyana ki bhaugolika sthiti ki mahatvapurna visheshataem haim : pashchima mem banjara aura purva mem halona | ye donom nadiyam narmada ki sahayaka nadiyam haim, jo ki udyana ke pashchimi praveshadvara ke 64 kilomitara (40 mila) uttara-pashchima mem mandala ke jila mukhyalaya nagara se hokara bahati hai | kanha ki ghatiyam sthaniya bhasha mem dadara kahalane vale patharom vale pahaड़om se samavarita haim | prabandhana ke uddeshyom ke lie, udyana ko 5 kshetrom mem banta gaya hai : kisali, kanha aura mukki pashchimi khanda mem aura bhainsaghata aura supakhara purvi khanda mem | kisali, kanha aura mukki shrrinkhalaom ke andara, kuchha sada़kem aura sthana bhi paryatakom ke lie banda haim | yaha likhe jane taka (1998 madhya), kanha rashtriya udyana ke darshakom ke liye khule hue kshetra (227 varga kimi.) ka pratishata abhyantara kshetra ka 25% hai | kanha rashtriya udyana samanyataya uttama rakha-rakhava ki hui saड़kom ke vyapaka jala (700 kimi. se kinchita adhika) se juड़a hai |
kanha rashtriya udyana 1 navambara se 30 juna taka pratidina khula rahata hai, yadyapi prabandhana ke pasa juna anta ke purva varsha hone para jaldi banda karane ka vikalpa arakshita hai | kanha rashtriya udyana ke khulane ka samaya anumanatah mausami utara - chaढ़ava, satha hi vibhinna mausamom mem dina ki lambai ke sammata nirdharita kiya jata hai | kanha rashtriya udyana mem madhyanha avakasha sadaiva hota hai, taki karmachari bhojana aura vishrama kara sakem aura janavarom ko bhi thoड़a arama mila sake | 1000 isvi ke asa-pasa bane khajuraho (kha-ju-ra-ho) ke ye mandira vishva ke lie ratyatmaka prema ke bharata ke amara upahara haim | jivana ke rangina, sukhada, tathapi udatta sara ki murta rupa, khajuraho ki daiviya pratimaem svayam jivana ke liye upaharasvarupa haim | kisi samaya jumate hue khajura ke vrikshom, jinase khajuraho shahara ka namakarana hua, se abhirakshita yaha shahara kala aura murtishilpa ka adhyayana hai | vastava mem, chandela mandira aja bhi aghata, ashcharya, abhibhuti aura prashansa sahita hajaro anya bhavanaom ko jagane vale apane vividha akarshaka murtarupom sahita khajuraho ka kendra haim | hindu-arya (indo-aryana) vastukala ka eka utkrishta udaharana, mandira ki divarom para kala jatilata aura sadagi ka eka parishkrita sanyojana hai | khajuraho shahara apani utpatti 10vim shatabdi se batata hai jaba yaha chandela rajavansha ka gadha़ tha |
aja ka khajuraho vanijyikarana ke adyatana smarakom, samanya sabhyata ke pratikom - hotalom, bhojanalayom aura dukanom - se ज़yada parichita hai| mandirom ke rupa mem purna bhavyatashali itihasa apani saunya drishti se usa nagara ko dekha raha hai jo apane shasakom ki dena ke bala para aja taka jivanta hai|
nissandeha, yaha shahara pratyeka varsha marcha mem nrityotsava ke daurana jivanta ho uthata hai, jaba shastriya nritya ke purodha, biraju maharaja aura kelucharana mahapatra aura unake shishya mandira ki natakiya prishthabhumi mem prastuti dete haim | nissandeha, khajuraho eka aisa bharatiya gantavya nahim hai jise apa asani se chhoda़ sakate haim | chandela vastukala spashtatah apane charama para thi jaba ye mandira banae gae the | devom aura deviyom, shuravira yoddhaom, pashuom aura logom ke drishyom ko ankita karati ye murtiyam aura utkirnana bahudha premalapa mem tallina dikhai jati haim | kuchha ko abhibhuta aura anya ko aghati lagane vali vishaya vasana aura ratyatmakata ki yaha prastuti eka bara bhi ashlila nahim lagati | isa kala ki sundarata mandira kala rupa ki shuddhata ko samvriddha karati hai | ye mandira eka uttunga shikhara mem samapta hone vali (aisi) minarom ki eka shrrinkhala haim, jinamem se pratyeka anya ki apeksha adhika unchi hai | yaha ashcharyajanaka hai ki ye khanda ekadusare ke satha sahajata se juda़e hue haim jabaki inake banate samaya gara nahim tha | lakshmana mandira karigari mem jatila hai, kandariya mahadeva, jaham andaruni bhaga mem bhavyata se tarashi gayi 872 murtiyam haim, sabase baड़a aura vastushilpa ki drishti se sabase purna hai, mahadeva mandira sabase chhota hai; devi jagadamba mandira sarvaprathama vishnu ko samarpita tha, tatpashchata parvati aura anta mem kali ko samarpita kiya gaya| unnisavim shatabdi ke dusare ardhda mem banae gae sabase prabhavi mahalom mem se eka lalagadha़ mahala bikanera ke charom ora nishthura thara marusthala ke rukhe evam vishama satya ko nakarata hai| lalagadha़ mahala bharatiya-arabi shilpa ka sabase purna rupa se sanyukta udaharana tha| 
apa ganga goldana jubali nyujiyama mem pakki mitti ke samana, paramparagata rajasthani vadyayantra aura raॉyala bikanera rela ke eka chhote pratirupa, prachina galichom evam rajasi kapada़om ka ananda le sakate haim| 
badashaha akabara ke maranasanna hone ke karana rajakumara jahaॉngira dvara rayasinha ko dilli jane ke lie di ja rahi rajakiya ajna bhi dekha sakate haim| kemala bridinga pharma shahara se matra 8 kimi. dura hai isa sarakari unta palana kshetra mem apa ‘marusthala ke jahaja’ ke bare mem bahuta kuchha jana pate haim| 
apako pata lagata hai ki kana para balom vala lambe bali vala untha, bikanera ki prajati hai jo ki apani takata ke lie prasidhda hai| halake ranga vale untha jaisalamera se haim aura apani gati ke karana bahumulya haim|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

('rashtriya', 'udyana')
('kanha', 'rashtriya')
('vishnu', 'samarpita')
('kanha', 'mukki')
('lakshmana', 'mandira')
('madhava', 'rashtriya')
('indo', 'aryana')
('khajuraho', 'shahara')
('arya', 'indo')
('shastriya', 'nritya')
('jahangira', 'mahala')
('nritya', 'utsava')
('punya', 'sthala')
('phlevara', 'phuda')
('phuda', 'phestivala')
('raja', 'manasinha')
('antarika', 'bhaga')
('mahadeva', 'mandira')
('kisali', 'kanha')
('banae', 'gae')
('kali', 'samarpita')
('dakshina', 'bharata')
('raya', 'pravina')
('bundela', 'rajaputom')
('purvi', 'khanda')
('lalagadha़', 'mahala')
('5', 'kshetrom')
('uttara', 'pashchima')
('pashchima', 'mandala')
('jila', 'mukhyalaya')
('nagara', 'hokara')
('pahaड़om', 'samavarita')
('udyana', '5')
('patharom', 'pahaड़om')
('uddeshyom', 'udyana')
('sthaniya', 'bhasha')
('samavarita', 'prabandhana')
('bhasha', 'dadara')
('dadara', 'kahalane')
('kahalane', 'patharom')
('ghatiyam', 'sthaniya')
('bhojana', 'arama')
('mila', 'uttara')
('40', 'mila')
('vyutpatti', 'kanha

('murtarupom', 'sahita')
('chandela', 'mandira')
('mandira', 'aja')
('aja', 'aghata')
('aghata', 'ashcharya')
('ashcharya', 'abhibhuti')
('prashansa', 'sahita')
('sahita', 'hajaro')
('hajaro', 'anya')
('anya', 'bhavanaom')
('jagane', 'vividha')
('vividha', 'akarshaka')
('akarshaka', 'murtarupom')
('sahita', 'khajuraho')
('vishaya', 'vasana')
('hindu', 'arya')
('aryana', 'vastukala')
('vastukala', 'utkrishta')
('utkrishta', 'udaharana')
('udaharana', 'mandira')
('mandira', 'divarom')
('divarom', 'kala')
('kala', 'jatilata')
('parishkrita', 'sanyojana')
('utpatti', '10vim')
('10vim', 'shatabdi')
('batata', 'chandela')
('vastava', 'chandela')
('kala', 'murtishilpa')
('shahara', 'kala')
('abhirakshita', 'shahara')
('asa', 'pasa')
('pasa', 'bane')
('bane', 'khajuraho')
('khajuraho', 'kha')
('sthala', 'akarshanom')
('ju', 'ra')
('mandira', 'vishva')
('vishva', 'ratyatmaka')
('ratyatmaka', 'prema')
('prema', 'bharata')
('amara', 'upahara')
('rangina', 'sukhada')
('sukhada', 'tathapi')
('tatha

('pratyeka', 'adhyayana')
('adhyayana', 'yogya')
('sundara', 'udaharanom')
('ashoka', 'stambha')
('stambha', 'dakshina')
('dakshina', 'mukhya')
('nikata', 'sthita')
('gupta', 'mandira')
('mandira', '5vim')
('shatabdi', 'isvi')
('vastushilpa', 'shailiyom')
('bharata', 'mandira')
('mandira', 'vastukala')
('purane', 'jnata')
('jnata', 'udaharanom')
('bhopala', '68')
('68', 'kimi')
('dura', 'sanchi')
('sanchi', 'tisari')
('tisari', 'shatabdi')
('shatabdi', 'purva')
('sare', 'bauddha')
('bauddha', 'vastushilpiya')
('bauddha', 'kala')
('gae', 'praveshadvara')
('tarashe', 'gae')
('bhavyata', 'tarashe')
('prema', 'kahaniyom')
('13', 'vim')
('vim', 'shatabdi')
('shatabdi', 'hindu')
('hindu', 'paramara')
('paramara', 'rajaom')
('rajaom', 'mula')
('mula', 'rajadhani')
('rajadhani', 'malava')
('sultanom', 'jita')
('jita', 'liya')
('liya', 'dekhem')
('dekhem', 'mahala')
('mahala', 'snanagriha')
('snanagriha', 'kheme')
('kheme', 'pratyeka')
('pratyeka', 'aphagani')
('aphagani', 'vastushilpa')
('masj